In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tlc

In [3]:
dataset_name = "cats-and-dogs"
project_name = "image-classification-table"
table_name = "initial"

table = tlc.Table.from_names(    
    table_name="initial",
    dataset_name=dataset_name,
    project_name=project_name,
)

In [ ]:
from tools.split import split_table

splits = split_table(
    table=table,
    splits={"train": 0.7, "val": 0.2, "test": 0.1},
)

train_table = splits["train"]
val_table = splits["val"]
test_table = splits["test"]

In [ ]:
print(len(train_table))
print(len(val_table))
print(len(test_table))

In [ ]:
train_table.url